In [1]:
# import project

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [2]:
MODEL = "llama3.2"

# class for the website
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [3]:
system_prompt = "You are an assistant that analyzes the offers of two websites \
and provides a short summary for both sites, ignoring text that might be navigation related. \
Respond with the offers from both sites."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website1, website2):
    user_prompt = f"You are looking at two websites titled {website1.title}, {website2.title}"

    # 1st Website 
    user_prompt += f"### website one: {website1.title}\n"
    user_prompt += "The contents of this website are as follows;"
    user_prompt += "please provide a short summary of this website in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website1.text + "\n\n"

    # 2nd website
    user_prompt += f"### Website two: {website2.title}\n"
    user_prompt += "The contents of this website are as follow;"
    user_prompt += "please provide a short summary of this websire in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website2.text + "\n\n"

    
    # Compare the websites
    user_prompt += "Finally, compare the offers, announcements, or any relevant information "
    user_prompt += "From both websites and highlight key differences and provide which option is better from the two carriers.\n"

    return user_prompt
    


In [14]:
# See how this function creates exactly the format above

def messages_for(website1, website2):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website1, website2)}
    ]

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url1, url2):
    website1 = Website(url1)
    website2 = Website(url2)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website1, website2)
    )
    return response['message']['content']

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def displaySummary(url1, url2):
    # Generate the combined summary for both websites
    combined_summary = summarize(url1, url2)
    
    # Display the combined summary in Markdown format
    display(Markdown(combined_summary))

In [18]:
displaySummary("https://www.verizon.com/smartphones/apple-iphone-16-pro/",
    "https://www.t-mobile.com/cell-phone/apple-iphone-16-pro"
)

After analyzing the content of both websites, I've identified some key differences between T-Mobile's website and the other carrier's website (let's assume it's Verizon). Here are some highlights:

**T-Mobile Website**

1. **Trade-in program**: T-Mobile has a trade-in program that allows customers to upgrade their devices and receive credit towards a new device.
2. **Yearly Upgrade benefit**: T-Mobile offers a Yearly Upgrade benefit, which requires trading in a device that is in good condition. The carrier pays off 50% of the remaining balance on the trade-in device.
3. **Promotions**: T-Mobile frequently runs promotions and offers discounts on certain devices.
4. **JUMP! program**: T-Mobile's JUMP! program allows customers to upgrade their devices every year, without any additional fees.

**Verizon Website**

1. **Trade-in program**: Verizon has a trade-in program that also allows customers to upgrade their devices and receive credit towards a new device.
2. **Device upgrades**: Verizon offers a variety of devices with different pricing plans and features.
3. **JUMP Up plan**: Verizon's JUMP Up plan is similar to T-Mobile's Yearly Upgrade benefit, but it only pays off 25% of the remaining balance on the trade-in device.

**Key differences**

1. **Trade-in credit**: T-Mobile pays off 50% of the remaining balance on the trade-in device, while Verizon only pays off 25%.
2. **JUMP! program**: T-Mobile's JUMP! program allows customers to upgrade their devices every year, without any additional fees. Verizon's JUMP Up plan requires customers to pay a $10 monthly fee for the first year.
3. **Device options**: Verizon offers a wider range of devices with different pricing plans and features.

**Which option is better?**

Based on my analysis, T-Mobile's website appears to offer more competitive trade-in credits and a more flexible JUMP! program. The carrier also frequently runs promotions and offers discounts on certain devices. However, Verizon's website has a wider range of devices available with different pricing plans and features.

Ultimately, the best option for an individual customer will depend on their specific needs and preferences. If you're looking for a flexible trade-in program and a low-cost device upgrade option, T-Mobile may be the better choice. However, if you prefer a wider range of devices and more flexibility in terms of pricing plans and features, Verizon may be the better option.

Here's a summary of the key differences:

**T-Mobile**

* Trade-in credit: 50% off remaining balance
* JUMP! program: Upgrade every year without additional fees
* Promotions: Frequently run promotions and discounts on certain devices

**Verizon**

* Trade-in credit: 25% off remaining balance
* JUMP Up plan: Requires $10 monthly fee for the first year
* Device options: Wide range of devices with different pricing plans and features